<a href="https://colab.research.google.com/github/Duks31/google-colab-code/blob/main/mnist_tensorflow_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.4 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa 
import math
from keras import layers 
from tensorflow import keras

(ds_train, ds_test), ds_info = tfds.load(
    "mnist", 
    split = ["train", "test"],
    shuffle_files = False,
    as_supervised = True, 
    with_info = True,  
    )

@tf.function
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label 

@tf.function
def rotate(img, max_degrees=25):
    degrees = tf.random.uniform([], -max_degrees, max_degrees,dtype=tf.float32)
    img = tfa.image.rotate(img, degrees*math.pi / 180, interpolation="BILINEAR")
    return img

@tf.function
def augment(image, label):
    image = tf.image.resize(image, size=[28, 28])
    image = rotate(image)
    
    # coloring of image 
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

def my_model():
    inputs = keras.Input(shape=(28, 28, 1))
    x = layers.Conv2D(32, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x) 
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    return keras.Model(inputs=inputs, outputs=outputs)

model = my_model()
# compile model 
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"],
)
# model.fit
model.fit(ds_train, epochs=20, verbose=2)
# model.evaluate
model.evaluate(ds_test)
# after training convert to tflite model
model.save("model")

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Epoch 1/20
1875/1875 - 30s - loss: 0.2846 - accuracy: 0.9213 - 30s/epoch - 16ms/step
Epoch 2/20
1875/1875 - 13s - loss: 0.0855 - accuracy: 0.9745 - 13s/epoch - 7ms/step
Epoch 3/20
1875/1875 - 14s - loss: 0.0611 - accuracy: 0.9817 - 14s/epoch - 8ms/step
Epoch 4/20
1875/1875 - 13s - loss: 0.0501 - accuracy: 0.9851 - 13s/epoch - 7ms/step
Epoch 5/20
1875/1875 - 13s - loss: 0.0441 - accuracy: 0.9870 - 13s/epoch - 7ms/step
Epoch 6/20
1875/1875 - 13s - loss: 0.0383 - accuracy: 0.9880 - 13s/epoch - 7ms/step
Epoch 7/20
1875/1875 - 13s - loss: 0.0364 - accuracy: 0.9888 - 13s/epoch - 7ms/step
Epoch 8/20
1875/1875 - 13s - loss: 0.0324 - accuracy: 0.9902 - 13s/epoch - 7ms/step
Epoch 9/20
1875/1875 - 13s - loss: 0.0300 - accuracy: 0.9910 - 13s/epoch - 7ms/step
Epoch 10/20
1875/1875 - 13s - loss: 0.0279 - accuracy: 0.9915 - 13s/epoch - 7ms/step
Epoch 11/20
1875/1875 - 13s - loss: 0.0

In [7]:
!zip -r /content/model.zip /content/model

  adding: content/model/ (stored 0%)
  adding: content/model/assets/ (stored 0%)
  adding: content/model/saved_model.pb (deflated 89%)
  adding: content/model/fingerprint.pb (stored 0%)
  adding: content/model/variables/ (stored 0%)
  adding: content/model/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: content/model/variables/variables.index (deflated 67%)
  adding: content/model/keras_metadata.pb (deflated 92%)


In [9]:
import tensorflow as tf 

saved_dir = "/content/model"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
model_lite = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(model_lite)